In [ ]:
# Import required packages
import requests
import pandas as pd
from datetime import datetime
from pytz import timezone
import config

In [ ]:
# Define variables for our API key, AOI, and list of satellites
api_key = config.api_key # Insert your own api code here
bbox = '-6.15450317,53.30408682,-6.07896861,53.34080955'  # AOI bounding box. Dublin Bay in this case
satellites = 'Sentinel-1A, Sentinel-1B, Sentinel-2A, Sentinel-2B, Sentinel-3A, Sentinel-3B, Landsat-8'

In [ ]:
# Construct the API request URL using the variables above and send the request
url = f'https://api.spectator.earth/overpass/?api_key={api_key}&bbox={bbox}&satellites={satellites}'
response = requests.get(url)
data = response.json()

In [ ]:
# Extract date and satellite name from the response and print
overpasses = [{'date': i['date'], 'satellite': i['satellite']} for i in data['overpasses']]
overpasses

In [ ]:
# Create a dataframe with results and convert dates
data = pd.DataFrame(overpasses)
data['datetime'] = pd.to_datetime(data['date'])
data['day'] = data['datetime'].dt.day_name()
data['date'] = data['datetime'].dt.date
data.head()

In [ ]:
# Convert datetime to Irish Standard Time (IST) and Coordinated Universal Time (UTC) and add to dataframe
ist_timezone = timezone('Europe/Dublin')
data['ist'] = data['datetime'].apply(lambda x: x.astimezone(ist_timezone).strftime('%H:%M:%S'))
data['utc'] = data['datetime'].dt.strftime('%H:%M:%S')
data.drop(columns=['datetime'], axis=1, inplace=True) # datetime column no longer needed
data.head()

In [ ]:
# Export to CSV
data.to_csv('output/sample.csv', index=False)